In [1]:
import pandas as pd

In [76]:
from google.colab import files
uploaded = files.upload()

Saving spider-man-1_reviews.csv to spider-man-1_reviews.csv
Saving spider-man-2_reviews.csv to spider-man-2_reviews.csv
Saving spider-man-3_reviews.csv to spider-man-3_reviews.csv


In [60]:
# Read data, organize with new uniform headers
headerList = ['movie', 'user', 'date', 'title', 'review', 'rating']
file1 = pd.read_csv('spider-man-1_reviews.csv', names=headerList)
file2 = pd.read_csv('spider-man-2_reviews.csv', names=headerList)
file3 = pd.read_csv('spider-man-3_reviews.csv', names=headerList)

In [65]:
# Create a single large dataframe from all files and drop rows with missing values
df = pd.concat([file1,file2,file3]).dropna()

In [66]:
df

,movie,user,date,title,review,rating
0,Spider-Man (2002),jaredpahl,5 June 2016,Iconic Electrifying Classy Vibrant and Emotion...,The world of movie superheroes began with Supe...,10.0
1,Spider-Man (2002),ronbell-23984,27 October 2020,Way better than the recent reboots,Really good movie I enjoyed the whole Trilogy ...,9.0
2,Spider-Man (2002),nickmesafilms,19 May 2012,Spider-Man (2002) - With Great Power Comes Gre...,Considered as one of the most successful super...,9.0
3,Spider-Man (2002),safenoe,25 February 2022,Pre MCU,Before the Marvel Cinematic Universe was launc...,10.0
4,Spider-Man (2002),Smells_Like_Cheese,5 May 2002,One of the best movies of the year!!!,Spider-Man is seriously the best movie of the ...,10.0
...,...,...,...,...,...,...
2184,Spider-Man 3,sjchelsea,5 May 2007,All good things must come to an end...,The first two entries into this franchise were...,3.0
2185,Spider-Man 3,bson3605,10 May 2007,Great movie with spectacular action and a good...,I just saw the movie and thought it was great....,10.0
2186,Spider-Man 3,deenimuk,6 May 2007,nice fit as a trilogy,OK by now you have read several reviews. My op...,6.0
2187,Spider-Man 3,Jrbirdman77,12 November 2007,Please sir I want some more.,I think we've said all there is to say about t...,6.0


In [73]:
# Remove unnecessary data (keeping review and rating)
df = df.drop(['movie','user','date','title'], axis=1)

In [74]:
# Remove 'neutral' scores (ratings of 5 and 6)
df = df.drop(df[(df.rating < 7) & (df.rating > 4)].index)

In [75]:
# Inspect data
df

,review,rating
0,The world of movie superheroes began with Supe...,10.0
1,Really good movie I enjoyed the whole Trilogy ...,9.0
2,Considered as one of the most successful super...,9.0
3,Before the Marvel Cinematic Universe was launc...,10.0
4,Spider-Man is seriously the best movie of the ...,10.0
...,...,...
2182,For the most part this movie was great. Plenty...,8.0
2183,So after waiting a long time to watch the new ...,7.0
2184,The first two entries into this franchise were...,3.0
2185,I just saw the movie and thought it was great....,10.0
